<a href="https://colab.research.google.com/github/jphall663/GWU_ML/blob/main/notebook/lecture_7/Assignment_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# License 
***
Copyright (C) 2017 -- 2022 J. Patrick Hall, jphall@gwu.edu

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

1. Standard Python imports and config

In [ ]:
# imports
import numpy as np
import pandas as pd

# to upload local files
import io
from google.colab import files  

# display matplotlib graphics in notebook
%matplotlib inline

# random seed for better reproducibility
SEED = 12345

2. Install Java

In [ ]:
# install Java for h2o backend
!apt-get install default-jre
!java -version

3. Install h2o

In [ ]:
# install h2o
!pip install h2o 

4. Import h2o package and required classes

In [ ]:
# import h2o
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator 
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch

5. Start h2o Server

In [ ]:
# start and connect to h2o server
h2o.init()

6. Upload example data

In [ ]:
# special google collab command to upload a file from computer
uploaded = files.upload() # REQUIRES STUDENT INPUT

In [ ]:
# 7
uploaded.keys() # what is stored in that Python object?

8. Covert to Pandas DataFrame

In [ ]:
# convert data to Pandas DataFrame
raw = pd.read_csv(io.StringIO(uploaded['loan_clean.csv'].decode('utf-8'))) # name in quotes here must match name above

9. Convert to H2O Frame

In [ ]:
frame = h2o.H2OFrame(raw) 

10. Split into train, validation, and test partitions -- with seed!!

In [ ]:
# split into 40% training, 30% validation, and 30% test
train, valid, test = frame.split_frame([0.4, 0.3], seed=SEED) # REQUIRES STUDENT INPUT

11. Assign modeling roles

In [ ]:
# assign target and inputs
y = 'bad_loan'
X = [name for name in frame.columns if name not in ['id', '_WARN_', y, 'GRP_home_ownership']]
print(y)
print(X)
print(len(X))

12. Set measurment level across data partitions (just to be sure)

In [ ]:
# set target to factor - for binary classification
train[y] = train[y].asfactor()
valid[y] = valid[y].asfactor()
test[y] = test[y].asfactor()

13. Train a random forest -- grid search not needed

In [ ]:
# random forest

# initialize rf model
rf_model = H2ORandomForestEstimator(
    ntrees=500,                      # Up to 500 decision trees in the forest 
    max_depth=30,                    # trees can grow to depth of 30
    stopping_rounds=5,               # stop after validation error does not decrease for 5 iterations/new trees
    score_tree_interval=3,           # makes early stopping reproducible
    seed=SEED)                       # REQUIRES STUDENT INPUT

# train rf model
rf_model.train(
    x=X,
    y=y,
    training_frame=train,
    validation_frame=valid)

# print model information
rf_model

14. View random forest AUC across data partitions

In [ ]:
# measure rf AUC
print(rf_model.auc(train=True))
print(rf_model.auc(valid=True))
print(rf_model.model_performance(test_data=test).auc())

15. Calculate partial dependence for `STD_IMP_REP_int_rate`

In [ ]:
# partial dependence plots are an interpretation tool
pd0 = rf_model.partial_plot(data=train, cols=['STD_IMP_REP_int_rate'], plot=True)

16. Plot random forest feature importance

In [ ]:
_ = rf_model.varimp_plot()

17. Score new row of data

In [ ]:
# predict on new data -- REQUIRES STUDENT INPUT
new_row = pd.DataFrame.from_dict({
  "GRP_REP_home_ownership": [1],
  "GRP_addr_state": [20],
  "GRP_purpose": [1],
  "GRP_verification_status": [1],
  "STD_IMP_REP_annual_inc": [4],
  "STD_IMP_REP_delinq_2yrs": [1],
  "STD_IMP_REP_dti": [1],
  "STD_IMP_REP_emp_length": [2],
  "STD_IMP_REP_int_rate": [1],
  "STD_IMP_REP_loan_amnt": [1],
  "STD_IMP_REP_longest_credit_lengt": [1],
  "STD_IMP_REP_revol_util": [1],
  "STD_IMP_REP_term_length":[1],
  "STD_IMP_REP_total_acc": [0]},
  orient='columns')

# generate prediction -- REQUIRES STUDENT INPUT
rf_model.predict(h2o.H2OFrame(new_row))

18. Train GBM with random grid search

In [ ]:
# GBM with random hyperparameter search
# train many different GBM models with random hyperparameters
# and select best model based on validation error

# define random grid search parameters
hyper_parameters = {'ntrees':list(range(0, 500, 50)),
                    'max_depth':list(range(0, 20, 2)),
                    'sample_rate':[s/float(10) for s in range(1, 11)],
                    'col_sample_rate':[s/float(10) for s in range(1, 11)]}

# define search strategy
search_criteria = {'strategy':'RandomDiscrete',
                   'max_models':20,
                   'max_runtime_secs':600,
                   'seed': SEED} # REQUIRES STUDENT INPUT

# initialize grid search
gsearch = H2OGridSearch(H2OGradientBoostingEstimator,
                        hyper_params=hyper_parameters,
                        search_criteria=search_criteria)

# execute training w/ grid search
gsearch.train(x=X,
              y=y,
              training_frame=train,
              validation_frame=valid,
              seed=SEED) # REQUIRES STUDENTS INPUT

19. View grid search results

In [ ]:
# show grid search results
gsearch.show()

# select best model
gbm_model = gsearch.get_grid()[0]

# print model information
gbm_model

20. View GBM AUC across data partitions

In [ ]:
# measure gbm AUC
print(gbm_model.auc(train=True))
print(gbm_model.auc(valid=True))
print(gbm_model.model_performance(test_data=test).auc())

21. Calculate partial dependence for `STD_IMP_REP_int_rate`

In [ ]:
# partial dependence plots are an interpretation tool
pd1 = gbm_model.partial_plot(data=train, cols=['STD_IMP_REP_int_rate'], plot=True)

22. Plot GBM feature importance

In [ ]:
_ = gbm_model.varimp_plot()

23. Score new row of data

In [ ]:
# predict on new data -- REQUIRES STUDENT INPUT
new_row = pd.DataFrame.from_dict({
  "GRP_REP_home_ownership": [1],
  "GRP_addr_state": [20],
  "GRP_purpose": [1],
  "GRP_verification_status": [1],
  "STD_IMP_REP_annual_inc": [4],
  "STD_IMP_REP_delinq_2yrs": [1],
  "STD_IMP_REP_dti": [1],
  "STD_IMP_REP_emp_length": [2],
  "STD_IMP_REP_int_rate": [1],
  "STD_IMP_REP_loan_amnt": [1],
  "STD_IMP_REP_longest_credit_lengt": [1],
  "STD_IMP_REP_revol_util": [1],
  "STD_IMP_REP_term_length":[1],
  "STD_IMP_REP_total_acc": [0]},
  orient='columns')

# generate prediction -- REQUIRES STUDENT INPUT
gbm_model.predict(h2o.H2OFrame(new_row))

24. Shutdown `h2o`

In [ ]:
# shutdown h2o
h2o.cluster().shutdown(prompt=False)